# IMT574 Problem Set 3: Causality (100pt)

## Name: Ganapathy S L

### Import the libraries

In [160]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import scipy.stats as stats
import statsmodels.formula.api as smf

%matplotlib inline
sb.set()

In [161]:
prog = pd.read_csv('progresa-sample.csv.bz2')
prog.shape

(77250, 21)

### 1. Descriptive Analysis

#### 1.1. Summary Statistics

In [162]:
# Checking for missing values
null = pd.DataFrame(prog.isnull().sum().reset_index(name = 'Null'))

In [163]:
prog['poor'] = prog['poor'].map({'pobre': 1, 'no pobre': 0})
prog['progresa'] = prog['progresa'].map({'basal': 1, '0': 0})

In [164]:
# Dropping village, folnum, year
prog_new = prog.drop(['village', 'folnum', 'year'], axis=1)

#Calculating the mean and std. dev for the numerical variables
prog_stat = pd.DataFrame(pd.merge(prog_new.mean().reset_index(name = 'Mean'), 
                      prog_new.std().reset_index(name = 'Std Dev'), 
                      on = ['index'])).sort_values(by = 'index')

In [165]:
pd.DataFrame(pd.merge(prog_stat, null, on = ['index']))

,index,Mean,Std Dev,Null
0,age,11.366460,3.167744,0
1,dist_cap,147.674452,76.063134,0
2,dist_sec,2.418910,2.234109,0
3,fam_n,7.215715,2.352900,0
4,grc,3.963537,2.499063,6549
5,grc97,3.705372,2.572387,0
6,hohage,44.436717,11.620372,10
7,hohedu,2.768104,2.656106,0
8,hohsex,0.925185,0.263095,20
9,hohwag,586.985312,788.133664,0


We have a maximum of 8543 missing values in sc column. Comparing the number of observations in the data it is very less ((8549/77250) = 11%). We can usually impute the data with mean or median or remove the observations depending on the circumstances of the data collected. Since removing is not the best option, imputing the missing data might impact on what the data might tell. Thus, for this analysis we will be ignoring the null values.

#### 1.2. Differences at Baseline

In [166]:
# Separate the treatement and control group in year 97

treatment_97 = prog[(prog.year == 97) & (prog.poor == 1) & (prog.progresa == 1)]
control_97 = prog[(prog.year == 97) & (prog.poor == 1) & (prog.progresa == 0)]

# Dropping the variables
treatment_97 = treatment_97.drop(['year', 'poor', 'progresa', 'village', 'folnum'], axis = 1)
control_97 = control_97.drop(['year', 'poor', 'progresa', 'village', 'folnum'], axis = 1)

In [167]:
t_stats = pd.DataFrame(pd.merge(treatment_97.mean().reset_index(name = 'Avg(Treatment)'), 
                      control_97.mean().reset_index(name = 'Avg(Control)'), 
                      on = ['index']))

t_stats['Difference'] = t_stats['Avg(Treatment)'] - t_stats['Avg(Control)']

p_value = []

for i in treatment_97.columns:
    p_value.append(stats.ttest_ind(treatment_97[i], control_97[i], nan_policy = 'omit').pvalue)

t_stats['p-value'] = p_value
t_stats['significance'] = t_stats['p-value'] < 0.05
t_stats.sort_values(by = 'index')

,index,Avg(Treatment),Avg(Control),Difference,p-value,significance
13,age,10.716991,10.742023,-0.025032,4.785594e-01,False
7,dist_cap,150.829074,153.769730,-2.940656,8.415005e-04,True
2,dist_sec,2.453122,2.507662,-0.054540,3.569843e-02,True
5,fam_n,7.281327,7.302469,-0.021142,4.271039e-01,False
4,grc,3.531599,3.543050,-0.011450,6.890151e-01,False
14,grc97,3.531599,3.543050,-0.011450,6.890151e-01,False
12,hohage,43.648828,44.276918,-0.628090,1.796243e-06,True
8,hohedu,2.663139,2.590348,0.072791,1.105093e-02,True
11,hohsex,0.924656,0.922947,0.001709,5.711858e-01,False
9,hohwag,544.339544,573.163558,-28.824015,3.253835e-04,True


#### 3. Are there statistically significant differences between treatment and control villages as baseline?

From the t-statistics, we calculate whether the difference between the control and the treatment group with various factors are having any statistical significance that we can reject the null hypothesis that there is no difference between the two groups. Here we can understand from the above table that for some variables the difference between the treatment and control group is statistically significant even before the treatment has started. For example, the villages in the treatment group were certainly much closer to the capital and they also had the closer proximity to a secondary school. But there are certainly other factors too such as the treatment group has much worser welfare index than the control group and they also make less wages compared to control group.

#### 4. Why does it matter if there are differences at baseline?

According to me, the differences in the baseline clearly indicates that where and why this program is being implemented and it shows us the motive of the program. The program is emphasized on developing the educational, nutritional and welfare development of the Mexico people and from the welfare_index, household educational index, houshold wage index differences between the treatment and control groups are significant that these differences matter and they imply the answer on why the program is implemented.

#### 5. What does this imply about how to measure the impact of the treatment?

This imply on we have to measure the impact of the treatment based the welfare_index, school enrollment, household wages. I think measuring these factors would show the impact of the treatment.

### 2. Measuring Impact

#### 1. Before-after estimator

A before-after estimator is to find the effect of the treatment based on the timeline that they have started. So we separate the data into two: Before the treatment has started, After the treatment has started and estimate the effect of treatment based on that.

In [179]:
# Adding a before column in the dataset
prog['before'] = (prog['year'] == 97)

In [181]:
prog[prog['poor'] == 1].groupby('before').sc.mean()

before
False    0.831730
True     0.819837
Name: sc, dtype: float64

From the above table, we could see a difference of 0.01 increase in the enrollment after the program has started.

#### Simple Linear Regression

In [184]:
m11 = smf.ols(formula = 'sc ~ before', data = prog[prog['poor'] == 1])
r11 = m1.fit()
r11.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     sc   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     14.28
Date:                Wed, 05 Feb 2020   Prob (F-statistic):           0.000158
Time:                        00:50:31   Log-Likelihood:                -26278.
No. Observations:               58372   AIC:                         5.256e+04
Df Residuals:                   58370   BIC:                         5.258e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.8317      0.002    363.057      0.000       0.827       0.836
before[T.True]    -0.0119      0.003     -3.779      0.000      -0.018      -0.006
==============================================================================
Omnibus:                    15383.245   Durbin-Watson:                   1.397
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            30726.067
Skew:                          -1.714   Prob(JB):                         0.00
Kurtosis:                       3.939   Cond. No.                         2.69
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The simple linear regression model also implies the same. It exactly shows us the effect of the program through the coefficient of the before variable which is 0.0119 which is the difference that we mentioned in the previous block. 

#### Multiple Linear Regression

In [186]:
m12 = smf.ols(formula = 'sc ~ before + dist_cap + dist_sec + hohage + hohedu + hohwag + min_dist + sex + welfare_index', 
             data = prog[prog['poor'] == 1])
r12 = m12.fit()
r12.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     sc   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     187.2
Date:                Wed, 05 Feb 2020   Prob (F-statistic):               0.00
Time:                        00:50:58   Log-Likelihood:                -25340.
No. Observations:               58192   AIC:                         5.070e+04
Df Residuals:                   58182   BIC:                         5.079e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.9146      0.012     77.854      0.000       0.892       0.938
before[T.True]    -0.0110      0.003     -3.551      0.000      -0.017      -0.005
dist_cap           0.0002   2.75e-05      8.053      0.000       0.000       0.000
dist_sec          -0.0086      0.001    -11.944      0.000      -0.010      -0.007
hohage            -0.0023      0.000    -15.331      0.000      -0.003      -0.002
hohedu             0.0111      0.001     16.217      0.000       0.010       0.012
hohwag         -5.627e-07   2.23e-06     -0.252      0.801   -4.94e-06    3.82e-06
min_dist           0.0004   5.08e-05      7.936      0.000       0.000       0.001
sex                0.0289      0.003      9.321      0.000       0.023       0.035
welfare_index     -0.0001   1.41e-05     -8.763      0.000      -0.000   -9.58e-05
==============================================================================
Omnibus:                    14484.417   Durbin-Watson:                   1.422
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            27943.776
Skew:                          -1.640   Prob(JB):                         0.00
Kurtosis:                       3.877   Cond. No.                     7.85e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.85e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In the multiple regression, we try to see the effect of other variables whether they have impact on the increase on the educational growth in the poor households. We could see that the effect of before variable has the same as before simple linear regression. But it also shows that other variables too have an impact in the increase in the secondary school enrollment. 

#### 2. Cross-sectional estimator

In cross-sectional estimator we try to estimate the effect of the treatment by selecting only the data after the program has started. Our goal is to estimate that the effect is only because of the treatment.

In [197]:
prog[(prog['before'] == 0) & (prog['poor'] == 1)].groupby('progresa').sc.mean()

progresa
0    0.807637
1    0.846479
Name: sc, dtype: float64

From the above difference table we could that the education enrollment is higher in the treatment group comapred to the control group with a difference of 0.039.

In [196]:
#### Simple Linear Regression
m21 = smf.ols(formula = 'sc ~ progresa', data = prog[(prog['before'] == 0) & (prog['poor'] == 1)])
r21 = m21.fit()
r21.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     sc   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     69.87
Date:                Wed, 05 Feb 2020   Prob (F-statistic):           6.64e-17
Time:                        01:01:58   Log-Likelihood:                -11926.
No. Observations:               27450   AIC:                         2.386e+04
Df Residuals:                   27448   BIC:                         2.387e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.8076      0.004    220.676      0.000       0.800       0.815
progresa       0.0388      0.005      8.359      0.000       0.030       0.048
==============================================================================
Omnibus:                     7638.939   Durbin-Watson:                   1.734
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            15767.534
Skew:                          -1.767   Prob(JB):                         0.00
Kurtosis:                       4.140   Cond. No.                         3.01
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The simple linear regression model also implies the same. It exactly shows us the effect of the program through the coefficient of the before variable which is 0.0388 which is the difference that we mentioned in the previous block.

In [198]:
# Multiple Linear Regression
# Selecting the variables that are statistically significant
m22 = smf.ols(formula = 'sc ~ progresa + dist_cap + dist_sec + hohage + hohedu + hohwag + min_dist + sex + welfare_index', 
             data = prog[(prog['before'] == 0) & (prog['poor'] == 1)])
r22 = m22.fit()
r22.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     sc   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     100.4
Date:                Wed, 05 Feb 2020   Prob (F-statistic):          1.52e-185
Time:                        01:04:52   Log-Likelihood:                -11451.
No. Observations:               27363   AIC:                         2.292e+04
Df Residuals:                   27353   BIC:                         2.300e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.8823      0.017     52.065      0.000       0.849       0.916
progresa          0.0337      0.005      7.318      0.000       0.025       0.043
dist_cap          0.0002   3.96e-05      6.025      0.000       0.000       0.000
dist_sec         -0.0096      0.001     -9.474      0.000      -0.012      -0.008
hohage           -0.0025      0.000    -11.197      0.000      -0.003      -0.002
hohedu            0.0111      0.001     11.283      0.000       0.009       0.013
hohwag        -2.228e-06   3.23e-06     -0.691      0.490   -8.55e-06    4.09e-06
min_dist          0.0004   7.29e-05      5.421      0.000       0.000       0.001
sex               0.0249      0.004      5.599      0.000       0.016       0.034
welfare_index -9.279e-05   2.02e-05     -4.592      0.000      -0.000   -5.32e-05
==============================================================================
Omnibus:                     7183.545   Durbin-Watson:                   1.761
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            14286.818
Skew:                          -1.688   Prob(JB):                         0.00
Kurtosis:                       4.068   Cond. No.                     7.87e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.87e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In the multiple regression, we try to see the effect of other variables whether they have impact on the increase on the educational growth in the poor households after the treatment has started. We could see that the effect of before variable has the same as before simple linear regression. But it also shows that other variables too have an impact in the increase in the secondary school enrollment.

#### 3. Differences-in-Differences Estimator

The DiD estimator tries to estimate the effect of the treatment by combining the cross sectional and before-after estimator.

In [200]:
prog[prog['poor'] == 1].groupby(['progresa', 'before']).sc.mean()

progresa  before
0         False     0.807637
          True      0.815186
1         False     0.846479
          True      0.822697
Name: sc, dtype: float64

Here we could see the effect of the treatment more clearly. In the control group, we could see that the schooling rate is dropping in after the treatment has started which might probably be due to various factors while in the treatment group we could see a considerable change of about 0.024 increase in the schooling rate. 

In [202]:
#### Simple Linear Regression
m31 = smf.ols(formula = 'sc ~ progresa * before', data = prog[prog['poor'] == 1])
r31 = m31.fit()
r31.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     sc   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     28.31
Date:                Wed, 05 Feb 2020   Prob (F-statistic):           2.76e-18
Time:                        11:21:48   Log-Likelihood:                -26242.
No. Observations:               58372   AIC:                         5.249e+04
Df Residuals:                   58368   BIC:                         5.253e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   0.8076      0.004    217.364      0.000       0.800       0.815
before[T.True]              0.0075      0.005      1.480      0.139      -0.002       0.018
progresa                    0.0388      0.005      8.233      0.000       0.030       0.048
progresa:before[T.True]    -0.0313      0.006     -4.835      0.000      -0.044      -0.019
==============================================================================
Omnibus:                    15346.988   Durbin-Watson:                   1.397
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            30608.651
Skew:                          -1.711   Prob(JB):                         0.00
Kurtosis:                       3.937   Cond. No.                         8.09
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

We try to estimate the same using a simple linear regression model and we could see that the regression model implies the same cross table that we have seen before.

In [199]:
# Multiple Linear Regression
m32 = smf.ols(formula = 'sc ~ before * progresa + dist_cap + dist_sec + hohage + hohedu + hohwag + min_dist + sex + \
welfare_index', data = prog[prog['poor'] == 1])
r32 = m32.fit()
r32.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     sc   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     158.1
Date:                Wed, 05 Feb 2020   Prob (F-statistic):               0.00
Time:                        01:05:35   Log-Likelihood:                -25314.
No. Observations:               58192   AIC:                         5.065e+04
Df Residuals:                   58180   BIC:                         5.076e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   0.8927      0.012     73.621      0.000       0.869       0.916
before[T.True]              0.0080      0.005      1.580      0.114      -0.002       0.018
progresa                    0.0338      0.005      7.229      0.000       0.025       0.043
before[T.True]:progresa    -0.0306      0.006     -4.778      0.000      -0.043      -0.018
dist_cap                    0.0002   2.76e-05      8.413      0.000       0.000       0.000
dist_sec                   -0.0086      0.001    -11.866      0.000      -0.010      -0.007
hohage                     -0.0023      0.000    -15.218      0.000      -0.003      -0.002
hohedu                      0.0111      0.001     16.160      0.000       0.010       0.012
hohwag                  -1.157e-07   2.23e-06     -0.052      0.959   -4.49e-06    4.26e-06
min_dist                    0.0004    5.1e-05      7.448      0.000       0.000       0.000
sex                         0.0287      0.003      9.267      0.000       0.023       0.035
welfare_index              -0.0001   1.41e-05     -8.671      0.000      -0.000   -9.44e-05
==============================================================================
Omnibus:                    14454.907   Durbin-Watson:                   1.422
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            27853.355
Skew:                          -1.637   Prob(JB):                         0.00
Kurtosis:                       3.874   Cond. No.                     8.28e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.28e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

We try to see the effect of other variables that created an impact on the enrollment of the students after the treatment has started and we could see that there is not much change in the coefficients of the variables compared to the linear regression but we could also see that certain variables are also causing similar impact comapred to the before and the progresa variables. 

#### 4. Compare the estimators

#### List the identifying assumptions (counterfactual assumptions) behind all three models. Which ones do you and more/less plausible?

In the before-after estimator, we could see that after the progresa program has started, there is a slight increase in the schooling rate (0.011) of the treatment group compared to the control group. This does not imply that this actually because of the program.

In the cross-sectional estimator, we selected the data after the treatmenthas started and we could see there is a considerable change between the treatment and control group (0.039) and we could say the difference is because of the treatment. But we don't have the implication on how the treatment and control group were before the treatment has started. Thus, we have to go to DiD estimator to properly estimate the effect of the program.

In the DiD estimator, we make the combination of the above two estimators and we could see how the treatment and control group where before and after the treatment has started. We added both the layers to see the effect of the program and it shows that there is some positive impact of the program in the treatment villages in the increase in the schooling rate.

According to me, for this analysis though DiD estimator has more variables to see the effect of the program, I think the cross-sectional estimator shows a better positive impact of the program in the treatment villages compared to the control villages. The reduction in schooling rate might be due to lot of various factors but this estimator shows the effect of the program better and I would choose cross-sectional in this case compared to DiD or before-after.

#### Compare the estimates of all three models. Do your analysis suggest that progresa program had a positive impact on schooling rates?

In all the three models, it shows that there is some positive impact of the program in the treatment villages but a single year of data would not suffice to arrive at a conclusion and having more observations for the next consecutive years and having much more different variables would help us to arrive at a conclusion. Also, we have estimated only how the schooling rate has been affected but we also have to see the welfare index, the nutrition factors which are also the motives of the program to estimate the positive effect of the program.